# Applying MHW defintion to NOW data

This has the code required to calculate the 90th percentile and climatology required by the Hobday (2016) paper.

Here we import the needed modules, and initialize a dask cluster. 

In [1]:
import xarray as xr
import now
import numpy as np
import time
from datetime import date
from matplotlib import pyplot as plt
from dask.distributed import LocalCluster, Client
import dask as da
import marineHeatWaves as mhw
%pylab inline
local_dir = "/g/data1a/e14/rm6294/dask-workers"
cluster = LocalCluster(processes=False, local_dir=local_dir)
client = Client(cluster)
client

Populating the interactive namespace from numpy and matplotlib


<Client: scheduler='inproc://10.0.64.28/28630/1' processes=1 cores=8>

In [ ]:
sst = xr.open_mfdataset('/g/data1a/e14/rm6294/matlab_trial/')

In [2]:
sst_reshaped = xr.open_zarr('/g/data1/e14/gs9353/POST-PROCESS/MHWs/SST_per_dayofyear_Present_vs_Future_50x50.zarr')['tos'].sel(simulation='Present')
'''
The following circularises the data, making rolling window processes possible. Guillaume pre did this for ^
sst_reshaped = sst_reshaped.isel(dayofyear = slice(0,-1)) # Removes day 366
start = sst_reshaped[:31]
start['dayofyear'] = range(366,397)
end = sst_reshaped[-31:]
end['dayofyear'] = range(-30, 1)
'''
circular_sst = sst_reshaped
#circular_sst = circular_sst.chunk({'x' : 30, 'y' : 30, 'dayofyear' : 31})

In [3]:
circular_sst

<xarray.DataArray 'tos' (dayofyear: 427, year: 19, y: 289, x: 431)>
dask.array<shape=(427, 19, 289, 431), dtype=float32, chunksize=(427, 19, 50, 50)>
Coordinates:
  * dayofyear   (dayofyear) int64 -30 -29 -28 -27 -26 ... 392 393 394 395 396
    nav_lat     (y, x) float32 dask.array<shape=(289, 431), chunksize=(50, 50)>
    nav_lon     (y, x) float32 dask.array<shape=(289, 431), chunksize=(50, 50)>
    simulation  <U7 'Present'
  * year        (year) int64 1990 1991 1992 1993 1994 ... 2005 2006 2007 2008
Dimensions without coordinates: y, x
Attributes:
    interval_operation:  360s
    interval_write:      1d
    long_name:           sea_surface_temperature
    online_operation:    average
    units:               degC

In [11]:
%%time
#Climatology 
raw_sst = circular_sst.mean('year')
sst_climatology_smooth_fat = raw_sst.rolling(dayofyear = 11, center = True).mean()
sst_climatology = sst_climatology_smooth_fat.rolling(dayofyear = 31, center = True).mean()

/g/data1/e14/rm6294/miniconda3/envs/now-postprocess/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: divide by zero encountered in true_divide
  x = np.divide(x1, x2, out)
/g/data1/e14/rm6294/miniconda3/envs/now-postprocess/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/g/data1/e14/rm6294/miniconda3/envs/now-postprocess/lib/python3.6/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/g/data1/e14/rm6294/miniconda3/envs/now-postprocess/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: divide by zero encountered in true_divide
  x = np.divide(x1, x2, out)
/g/data1/e14/rm6294/miniconda3/envs/now-postprocess/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/g/data1/e14/rm6294/

CPU times: user 1min 30s, sys: 2min 23s, total: 3min 54s
Wall time: 53.2 s


In [12]:
%%time
sst_climatology = sst_climatology.isel(dayofyear = slice(31,-31))
sst_climatology.to_netcdf('/g/data1a/e14/rm6294/NOWMHW/orig/pastClim.nc')

/g/data1/e14/rm6294/miniconda3/envs/now-postprocess/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: divide by zero encountered in true_divide
  x = np.divide(x1, x2, out)
/g/data1/e14/rm6294/miniconda3/envs/now-postprocess/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


CPU times: user 51.2 s, sys: 1min 46s, total: 2min 37s
Wall time: 1min 18s


In [4]:
#Threshold. Same here, no slicing:
percRolling = circular_sst.rolling(dayofyear=11, center=True).construct('rolling_days')

# This takes in the sst, performs the percentile calculation, then creates a new dimension and coordinate names 
# to prepapre the data for the final output
stacked = percRolling.stack(z = ('rolling_days', 'year'))
rawPerc_data = da.array.apply_along_axis(np.nanpercentile, stacked.get_axis_num('z'), stacked.data, 90)
sst_coords = circular_sst.coords
new_coords = {name : sst_coords[name] for name in sst_coords if name != 'year'}
new_dims = [name for name in circular_sst.dims if name != 'year']
rawPerc = xr.DataArray(rawPerc_data, coords = new_coords, dims = new_dims)

In [5]:
%%time
rawPerc.to_netcdf('/g/data1a/e14/rm6294/NOWMHW/orig/rawPercentiles.nc')

/g/data1/e14/rm6294/miniconda3/envs/now-postprocess/lib/python3.6/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


CPU times: user 5h 41min 16s, sys: 3h 7min 40s, total: 8h 48min 56s
Wall time: 5h 35min 14s


In [6]:
%%time
#Final rolling mean over a month (smoothing)
rawPerc = xr.open_dataset('/g/data1a/e14/rm6294/NOWMHW/orig/rawPercentiles.nc')
sst_threshold = rawPerc.rolling(dayofyear=31, center = True).mean()
print("Data smoothed, DONE.")
sst_threshold = sst_threshold.isel(dayofyear = slice(31,-31))
sst_threshold.to_netcdf('/g/data1a/e14/rm6294/NOWMHW/orig/pastThresh.nc')

/g/data1/e14/rm6294/miniconda3/envs/now-postprocess/lib/python3.6/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


Data smoothed, DONE.
CPU times: user 18 s, sys: 13.4 s, total: 31.4 s
Wall time: 16.5 s
